In [1]:
# 필요한 라이브러리 불러오기

from konlpy.tag import Mecab 

import pandas as pd
import numpy as np

from collections import Counter

import warnings
warnings.filterwarnings('ignore')

dir = 'C:/Users/juwan/Desktop/산학프로젝트/using_data/'

In [2]:
# mecab 실행

mecab = Mecab(dicpath=r"C:\Mecab\mecab-ko-dic") # dictation path 설정 / mecab-ko-dic이 mecab의 단어사전임

In [3]:
# 형태소 분석할 데이터 준비 - 정책 내용 데이터 불러오기 

tagging = pd.read_csv(dir + 'content_tagging.csv', encoding = 'utf-8')

In [4]:
# name, description, content의 모든 내용으로 단어 추출

nouns_list = []

for i in range(len(tagging['content'])):
    nouns_list.extend(mecab.nouns(tagging['content'][i]))

In [5]:
# 불용어 제거 - 2글자 이상인 단어만 추출

nouns_list = [i for i in nouns_list if len(i) >= 2]

In [6]:
# 행 전체 다 보기

pd.set_option('display.max_rows',None)

In [ ]:
tagging.head()

In [13]:
# 각 카테고리별 데이터 프레임 만들기

money_df = tagging.loc[tagging['money'] == 1].reset_index(drop=True)
interview_df = tagging.loc[tagging['interview'] == 1].reset_index(drop=True)
edu_df = tagging.loc[tagging['edu'] == 1].reset_index(drop=True)
job_df = tagging.loc[tagging['job'] == 1].reset_index(drop=True)
consult_df = tagging.loc[tagging['consult'] == 1].reset_index(drop=True)

In [14]:
# 각 카테고리별 단어들 리스트 생성

nouns_money = []
for i in range(len(money_df['content'])):
    nouns_money.extend(mecab.nouns(money_df['content'][i]))
    
nouns_interview = []
for i in range(len(interview_df['content'])):
    nouns_interview.extend(mecab.nouns(interview_df['content'][i]))

nouns_edu = []
for i in range(len(edu_df['content'])):
    nouns_edu.extend(mecab.nouns(edu_df['content'][i]))
    
nouns_job = []
for i in range(len(job_df['content'])):
    nouns_job.extend(mecab.nouns(job_df['content'][i]))
    
nouns_consult = []
for i in range(len(consult_df['content'])):
    nouns_consult.extend(mecab.nouns(consult_df['content'][i]))

In [17]:
# 불용어 똑같이 제거 (2글자 이상인 단어)

nouns_money = [i for i in nouns_money if len(i) >= 2]
nouns_interview = [i for i in nouns_interview if len(i) >= 2]
nouns_edu = [i for i in nouns_edu if len(i) >= 2]
nouns_job = [i for i in nouns_job if len(i) >= 2]
nouns_consult = [i for i in nouns_consult if len(i) >= 2]

In [18]:
print(len(nouns_money))
print(len(nouns_interview))
print(len(nouns_edu))
print(len(nouns_job))
print(len(nouns_consult))

# 다해서 61162

15855
3781
16411
14446
10669


In [22]:
# 각 카테고리별 단어들 리스트 합친것 (중복 매우 많음, 다해서 61162개의 단어)

nouns = nouns_money + nouns_interview + nouns_edu +  nouns_job + nouns_consult

In [ ]:
# 전체 정책의 단어들의 빈도수 출력

cnt_df0 = pd.DataFrame()
cnt_df0['description_word'] = dict(Counter(nouns_list)).keys()
cnt_df0['total_cnt'] = dict(Counter(nouns_list)).values()

cnt_df0.head()

In [33]:
### money/지원금 ### 카테고리의 단어들과 빈도수

cnt_rank = pd.DataFrame()
cnt_rank['description_word'] = pd.Series(list(dict(Counter(nouns_money)).keys()))
cnt_rank['cnt_money'] = pd.Series(list(dict(Counter(nouns_money)).values()))

cnt_rank = cnt_rank.sort_values(by='cnt_money', ascending = False)

### 전체 정책의 단어들의 빈도수 조인
cnt_rank = pd.merge(cnt_rank, cnt_df0, how = 'left', on = ['description_word'])

### 카테고리 안에서 단어 빈도수 / 전체 정책에서의 단어 빈도수
cnt_rank['pnt_money'] = cnt_rank['cnt_money'] / cnt_rank['total_cnt']

cnt_money_df = cnt_rank.copy() # 따로 저장

In [34]:
### interview/면접 ### 카테고리의 단어들과 빈도수

cnt_rank = pd.DataFrame()
cnt_rank['description_word'] = pd.Series(list(dict(Counter(nouns_interview)).keys()))
cnt_rank['cnt_interview'] = pd.Series(list(dict(Counter(nouns_interview)).values()))

cnt_rank = cnt_rank.sort_values(by='cnt_interview', ascending = False)

cnt_rank = pd.merge(cnt_rank, cnt_df0, how = 'left', on = ['description_word'])
cnt_rank['pnt_interview'] = cnt_rank['cnt_interview'] / cnt_rank['total_cnt']

cnt_interview_df = cnt_rank.copy()

In [35]:
### edu/교육 ### 카테고리의 단어들과 빈도수

cnt_rank = pd.DataFrame()
cnt_rank['description_word'] = pd.Series(list(dict(Counter(nouns_edu)).keys()))
cnt_rank['cnt_edu'] = pd.Series(list(dict(Counter(nouns_edu)).values()))

cnt_rank = cnt_rank.sort_values(by='cnt_edu', ascending = False)

cnt_rank = pd.merge(cnt_rank, cnt_df0, how = 'left', on = ['description_word'])
cnt_rank['pnt_edu'] = cnt_rank['cnt_edu'] / cnt_rank['total_cnt']

cnt_edu_df = cnt_rank.copy()

In [36]:
### job/일자리 ### 카테고리의 단어들과 빈도수

cnt_rank = pd.DataFrame()
cnt_rank['description_word'] = pd.Series(list(dict(Counter(nouns_job)).keys()))
cnt_rank['cnt_job'] = pd.Series(list(dict(Counter(nouns_job)).values()))

cnt_rank = cnt_rank.sort_values(by='cnt_job', ascending = False)

cnt_rank = pd.merge(cnt_rank, cnt_df0, how = 'left', on = ['description_word'])
cnt_rank['pnt_job'] = cnt_rank['cnt_job'] / cnt_rank['total_cnt']

cnt_job_df = cnt_rank.copy()

In [37]:
### consult/상담 ### 카테고리의 단어들과 빈도수

cnt_rank = pd.DataFrame()
cnt_rank['description_word'] = pd.Series(list(dict(Counter(nouns_consult)).keys()))
cnt_rank['cnt_consult'] = pd.Series(list(dict(Counter(nouns_consult)).values()))

cnt_rank = cnt_rank.sort_values(by='cnt_consult', ascending = False)

cnt_rank = pd.merge(cnt_rank, cnt_df0, how = 'left', on = ['description_word'])
cnt_rank['pnt_consult'] = cnt_rank['cnt_consult'] / cnt_rank['total_cnt']

cnt_consult_df = cnt_rank.copy()

### 카테고리 5개 모두 포함된 단어 제거

In [40]:
# 이너조인 - 5개 모두 포함되는 단어 추출

df = pd.merge(cnt_money_df, cnt_interview_df, on = 'description_word', how = 'inner')
df = pd.merge(df, cnt_edu_df, on = 'description_word', how = 'inner')
df = pd.merge(df, cnt_job_df, on = 'description_word', how = 'inner')
df = pd.merge(df, cnt_consult_df, on = 'description_word', how = 'inner')

In [ ]:
df = df[['description_word','cnt_money','pnt_money','cnt_interview','pnt_interview','cnt_edu','pnt_edu','cnt_job','pnt_job','cnt_consult','pnt_consult','total_cnt']]
pnt_df = df[['description_word','pnt_money','pnt_interview','pnt_edu','pnt_job','pnt_consult','total_cnt']]

pnt_df.head()

In [450]:
# cnt_money를 기준으로 가중치 곱해주기

pnt_df['pnt_interview'] = pnt_df['pnt_interview'] * (len(cnt_money) / len(cnt_interview))
pnt_df['pnt_edu'] = pnt_df['pnt_edu'] * (len(cnt_money) / len(cnt_edu))
pnt_df['pnt_job'] = pnt_df['pnt_job'] * (len(cnt_money) / len(cnt_job))
pnt_df['pnt_consult'] = pnt_df['pnt_consult'] * (len(cnt_money) / len(cnt_consult))

In [42]:
# pnt_df.to_csv('five_nouns.csv', index = False)

In [43]:
# 각 카테고리별로 pnt를 내림차순으로 정렬한 후 상위 20개 단어는 각 카테고리에 포함(그 이외는 삭제)

In [44]:
# 각 카테고리별 단어들 리스트 합친것 (중복 매우 많음, 다해서 61162개의 단어)

nouns = nouns_money + nouns_interview + nouns_edu + nouns_job + nouns_consult
len(nouns)

61162

In [ ]:
# 각 카테고리별 유니크 단어들로 데이터프레임 + 빈도수까지 추가

word_df = pd.DataFrame({'description_word' : list(set(nouns))})
word_df = pd.merge(word_df, cnt_df0, on = 'description_word', how='left')
word_df.head()

In [46]:
# 더미변수 생성

word_df['money'] = pd.Series([i in nouns_money for i in word_df['description_word']])
word_df['interview'] = pd.Series([i in nouns_interview for i in word_df['description_word']])
word_df['edu'] = pd.Series([i in nouns_edu for i in word_df['description_word']])
word_df['job'] = pd.Series([i in nouns_job for i in word_df['description_word']])
word_df['consult'] = pd.Series([i in nouns_consult for i in word_df['description_word']])

word_df = word_df.replace(True, 1).replace(False, 0)

In [48]:
# 5개 카테고리 모두 포함되는 단어 삭제

for i in range(len(word_df)):
    if word_df['money'][i] + word_df['interview'][i] + word_df['edu'][i] + word_df['job'][i] + word_df['consult'][i] == 5:
        word_df.drop(i, inplace = True)

In [57]:
# 5개 포함 단어 중 포함시킬 것들

five = pd.read_csv('five_nouns_add.csv', encoding = 'euc-kr')

In [50]:
# 5개 카테고리 모두 포함되는 단어 중 포함시킬 단어들 리스트로 만들기

add_list = list(five['pnt_money']) + list(five['pnt_interview']) + list(five['pnt_edu']) + list(five['pnt_job']) + list(five['pnt_consult'])

In [51]:
# 위 word_df와 같은 형식으로 추가할 단어들 데이터프레임 만들기

add_df = pd.DataFrame({'description_word' : list(set(add_list)), 'money':0,'interview':0,'edu':0,'job':0,'consult':0})
add_df = pd.merge(add_df,cnt_df0, on = 'description_word', how = 'left')

In [59]:
# 더미변수 채우기

for i in range(len(add_df)):
    if add_df['description_word'][i] in list(five['pnt_money']):
        add_df.iloc[i,1] = 1
        
    if add_df['description_word'][i] in list(five['pnt_interview']):
        add_df.iloc[i,2] = 1
        
    if add_df['description_word'][i] in list(five['pnt_edu']):
        add_df.iloc[i,3] = 1
        
    if add_df['description_word'][i] in list(five['pnt_job']):
        add_df.iloc[i,4] = 1
        
    if add_df['description_word'][i] in list(five['pnt_consult']):
        add_df.iloc[i,5] = 1

In [60]:
word_df = pd.concat([word_df, add_df]).reset_index(drop = True)

In [61]:
# word_df.to_csv('word_dataframe.csv', index = False)

### 카테고리 3, 4개 모두 포함된 단어 제거

### 4개 추출

In [ ]:
# 카테고리 4개에 포함되는 단어 추출

word_df_4 = word_df.loc[word_df['money'] + word_df['interview'] + word_df['edu'] + word_df['job'] + word_df['consult'] == 4]

In [76]:
# 카테고리 4개 겹치는 단어에 percent 붙이기

word_df_4 = pd.merge(word_df_4, cnt_money_df[['description_word','pnt_money']], how = 'left', on = 'description_word')
word_df_4 = pd.merge(word_df_4, cnt_interview_df[['description_word','pnt_interview']], how = 'left', on = 'description_word')
word_df_4 = pd.merge(word_df_4, cnt_edu_df[['description_word','pnt_edu']], how = 'left', on = 'description_word')
word_df_4 = pd.merge(word_df_4, cnt_job_df[['description_word','pnt_job']], how = 'left', on = 'description_word')
word_df_4 = pd.merge(word_df_4, cnt_consult_df[['description_word','pnt_consult']], how = 'left', on = 'description_word')

In [ ]:
word_df_4 = word_df_4[['description_word','pnt_money','pnt_interview','pnt_edu','pnt_job','pnt_consult','total_cnt']]

In [470]:
# word_df_4.to_csv('four_nouns.csv', index = False)

### 카테고리 3, 4개 모두 포함된 단어 제거

### 3개 추출

In [ ]:
# 카테고리 3개에 포함되는 단어 추출

word_df_3 = word_df.loc[word_df['money'] + word_df['interview'] + word_df['edu'] + word_df['job'] + word_df['consult'] == 3]

In [82]:
word_df_3 = pd.merge(word_df_3, cnt_money_df[['description_word','pnt_money']], how = 'left', on = 'description_word')
word_df_3 = pd.merge(word_df_3, cnt_interview_df[['description_word','pnt_interview']], how = 'left', on = 'description_word')
word_df_3 = pd.merge(word_df_3, cnt_edu_df[['description_word','pnt_edu']], how = 'left', on = 'description_word')
word_df_3 = pd.merge(word_df_3, cnt_job_df[['description_word','pnt_job']], how = 'left', on = 'description_word')
word_df_3 = pd.merge(word_df_3, cnt_consult_df[['description_word','pnt_consult']], how = 'left', on = 'description_word')

In [ ]:
word_df_3 = word_df_3[['description_word','pnt_money','pnt_interview','pnt_edu','pnt_job','pnt_consult','total_cnt']]
word_df_3.sort_values(by='total_cnt')

In [85]:
# word_df_3.to_csv('three_nouns.csv', index = False)

### 3,4 카테고리 들어간 단어 삭제, 살릴 단어만 추가

In [86]:
four = pd.read_csv('C:/Users/juwan/Desktop/산학프로젝트/four_nouns_add.csv', encoding = 'euc-kr')
three = pd.read_csv('C:/Users/juwan/Desktop/산학프로젝트/three_nouns_add.csv', encoding = 'euc-kr')

In [88]:
### 4개 카테고리 모두 포함되는 단어 삭제

for i in range(len(word_df)):
    if word_df['money'][i] + word_df['interview'][i] + word_df['edu'][i] + word_df['job'][i] + word_df['consult'][i] == 4:
        word_df.drop(i, inplace = True)

In [89]:
# 4개 카테고리 모두 포함되는 단어 중 포함시킬 단어들 리스트로 만들기

four_list = list(four['pnt_money']) + list(four['pnt_interview']) + list(four['pnt_edu']) + list(four['pnt_job']) + list(four['pnt_consult'])

In [90]:
add_df = pd.DataFrame({'description_word' : list(set(four_list)), 'money':0,'interview':0,'edu':0,'job':0,'consult':0})
add_df = pd.merge(add_df,cnt_df0, on = 'description_word', how = 'left')

In [91]:
add_df = add_df.dropna()
add_df = add_df.reset_index(drop = True)

In [92]:
# 더미변수 채우기

for i in range(len(add_df)):
    if add_df['description_word'][i] in list(four['pnt_money']):
        add_df.iloc[i,1] = 1
        
    if add_df['description_word'][i] in list(four['pnt_interview']):
        add_df.iloc[i,2] = 1
        
    if add_df['description_word'][i] in list(four['pnt_edu']):
        add_df.iloc[i,3] = 1
        
    if add_df['description_word'][i] in list(four['pnt_job']):
        add_df.iloc[i,4] = 1
        
    if add_df['description_word'][i] in list(four['pnt_consult']):
        add_df.iloc[i,5] = 1

In [93]:
word_df = pd.concat([word_df, add_df])

In [94]:
word_df = word_df.reset_index(drop = True)

In [ ]:
### 3개 카테고리 모두 포함되는 단어 삭제

for i in range(len(word_df)):
    if word_df['money'][i] + word_df['interview'][i] + word_df['edu'][i] + word_df['job'][i] + word_df['consult'][i] == 3:
        word_df.drop(i, inplace = True)

In [99]:
# 3개 카테고리 모두 포함되는 단어 중 포함시킬 단어들 리스트로 만들기

three_list = list(three['pnt_money']) + list(three['pnt_interview']) + list(three['pnt_edu']) + list(three['pnt_job']) + list(three['pnt_consult'])

In [100]:
add_df = pd.DataFrame({'description_word' : list(set(three_list)), 'money':0,'interview':0,'edu':0,'job':0,'consult':0})
add_df = pd.merge(add_df,cnt_df0, on = 'description_word', how = 'left')

In [101]:
add_df = add_df.dropna()
add_df = add_df.reset_index(drop = True)

In [102]:
# 더미변수 채우기

for i in range(len(add_df)):
    if add_df['description_word'][i] in list(three['pnt_money']):
        add_df.iloc[i,1] = 1
        
    if add_df['description_word'][i] in list(three['pnt_interview']):
        add_df.iloc[i,2] = 1
        
    if add_df['description_word'][i] in list(three['pnt_edu']):
        add_df.iloc[i,3] = 1
        
    if add_df['description_word'][i] in list(three['pnt_job']):
        add_df.iloc[i,4] = 1
        
    if add_df['description_word'][i] in list(three['pnt_consult']):
        add_df.iloc[i,5] = 1

In [103]:
word_df = pd.concat([word_df, add_df])

In [104]:
word_df = word_df.reset_index(drop = True)

In [110]:
# cnt행 붙여넣기

word_df_cnt = pd.merge(word_df, cnt_money_df[['description_word','cnt_money']], on = 'description_word', how = 'left')
word_df_cnt = pd.merge(word_df_cnt, cnt_interview_df[['description_word','cnt_interview']], on = 'description_word', how = 'left')
word_df_cnt = pd.merge(word_df_cnt, cnt_edu_df[['description_word','cnt_edu']], on = 'description_word', how = 'left')
word_df_cnt = pd.merge(word_df_cnt, cnt_job_df[['description_word','cnt_job']], on = 'description_word', how = 'left')
word_df_cnt = pd.merge(word_df_cnt, cnt_consult_df[['description_word','cnt_consult']], on = 'description_word', how = 'left')

In [111]:
# word_df에 포함되지 않은 단어 NA 없애기

word_df_cnt = word_df_cnt.fillna(0)

In [113]:
# 위에서 5,4,3카테고리 --> 2개로 바꿀 때 제외된 카테고리 0값 주기

for i in range(len(word_df_cnt)):
    if word_df_cnt['money'][i] == 0:
        word_df_cnt.iloc[i,7] = 0
    if word_df_cnt['interview'][i] == 0:
        word_df_cnt.iloc[i,8] = 0
    if word_df_cnt['edu'][i] == 0:
        word_df_cnt.iloc[i,9] = 0
    if word_df_cnt['job'][i] == 0:
        word_df_cnt.iloc[i,10] = 0
    if word_df_cnt['consult'][i] == 0:
        word_df_cnt.iloc[i,11] = 0

In [114]:
# pnt 다시 넣기

word_df_cnt['pnt_money'] = word_df_cnt['cnt_money'] / word_df_cnt['total_cnt']
word_df_cnt['pnt_interview'] = word_df_cnt['cnt_interview'] / word_df_cnt['total_cnt']
word_df_cnt['pnt_edu'] = word_df_cnt['cnt_edu'] / word_df_cnt['total_cnt']
word_df_cnt['pnt_job'] = word_df_cnt['cnt_job'] / word_df_cnt['total_cnt']
word_df_cnt['pnt_consult'] = word_df_cnt['cnt_consult'] / word_df_cnt['total_cnt']